# Enunciado


## Projeto - Extração de Dados I
------------------------------
## Sistema de Monitoramento de Avanços no Campo da Genômica  

## Contexto:  
O grupo trabalha no time de engenharia de dados na HealthGen, uma empresa especializada em genômica e pesquisa de medicina personalizada. A genômica é o estudo do conjunto completo de genes de um organismo, desempenha um papel fundamental na medicina personalizada e na pesquisa biomédica. Permite a análise do DNA para identificar variantes genéticas e mutações associadas a doenças e facilita a personalização de tratamentos com base nas características genéticas individuais dos pacientes.

A empresa precisa se manter atualizada sobre os avanços mais recentes na genômica, identificar oportunidades para pesquisa e desenvolvimento de tratamentos personalizados e acompanhar as tendências em genômica que podem influenciar estratégias de pesquisa e desenvolvimento. Pensando nisso, o time de dados apresentou uma proposta de desenvolvimento de um sistema que coleta, analisa e apresenta as últimas notícias relacionadas à genômica e à medicina personalizada, e também estuda o avanço do campo nos últimos anos.

O time de engenharia de dados tem como objetivo desenvolver e garantir um pipeline de dados confiável e estável. As principais atividades são:

### 1. Consumo de dados com a News API:  
Implementar um mecanismo para consumir dados de notícias de fontes confiáveis e especializadas em genômica e medicina personalizada, a partir da News API:  
https://newsapi.org/

### 2. Definir Critérios de Relevância:  
Desenvolver critérios precisos de relevância para filtrar as notícias. Por exemplo, o time pode se concentrar em notícias que mencionem avanços em sequenciamento de DNA, terapias genéticas personalizadas ou descobertas relacionadas a doenças genéticas específicas.

### 3. Cargas em Batches:  
Armazenar as notícias relevantes em um formato estruturado e facilmente acessível para consultas e análises posteriores. Essa carga deve acontecer 1 vez por hora. Se as notícias extraídas já tiverem sidos armazenadas na carga anterior, o processo deve ignorar e não armazenar as notícias novamente, os dados carregados não podem ficar duplicados.  
![Alt text](image.png)

### 4. Dados transformados para consulta do público final  
A partir dos dados carregados, aplicar as seguintes transformações e armazenar o resultado final para a consulta do público final:  

4.1 - Quantidade de notícias por ano, mês e dia de publicação;  

4.2 - Quantidade de notícias por fonte e autor;  

4.3 - Quantidade de aparições de 3 palavras chaves por ano, mês e dia de publicação (as 3 palavras chaves serão as mesmas usadas para fazer os filtros de relevância do item 2 (2. Definir Critérios de Relevância)).  

Atualizar os dados transformados 1 vez por dia.  

![Alt text](image-1.png)

----------------------------------------

Além das atividades principais, existe a necessidade de busca de dados por eventos em tempo real quando é necessário, para isso foi desenhado duas opções:

### Opção 1 - Apache Kafka e Spark Streaming:  

Preparar um pipeline com Apache Kafka e Spark Streaming para receber os dados do Produtor Kafka representado por um evento manual e consumir os dados com o Spark Streaming armazenando os resultados temporariamente. Em um processo paralelo, verificar os resultados armazenados temporiamente e armazenar no mesmo destino do item 3 (3. Cargas em Batches) aqueles resultados que ainda não foram armazenados no destino (os dados carregados não podem ficar duplicados). E por fim, eliminar os dados temporários após a verificação e a eventual carga.

![Alt text](image-2.png)

### Opção 2 - Webhooks com notificações por eventos:  
Configurar um webhook para adquirir as últimas notícias a partir de um evento representado por uma requisição POST e fazer a chamada da API e por fim armazenar os resultados temporariamente. Em um processo paralelo, verificar os resultados armazenados temporiamente e armazenar no mesmo destino do item 3 (3. Cargas em Batches) aqueles resultados que ainda não foram armazenados no destino (os dados carregados não podem ficar duplicados). E por fim, eliminar os dados temporários após a verificação e a eventual carga.

![Alt text](image-3.png)

Atividades que precisam ser realizadas pelo grupo definido em aula.  

O grupo precisa construir o pipeline de dados seguindo os requisitos das atividades principais e escolher entre a Opção 1 e Opção 2 para desenvolvimento.  

# Resolução

## 1 Bibliotecas
Para instalar as bibliotecas necessárias do projeto:

In [1]:
pip install requests newsapi-python

Para importas as bibliotecas e definir as variáveis necessárias:

In [2]:
from newsapi import NewsApiClient
import requests
import time
import pandas as pd
from datetime import datetime
import re
import os

API_KEY = '9a77398581d74beebbd29dbebd159a53'
PALAVRAS_CHAVES_GERAIS = '(genômica OR genômico) AND (terapia OR sequenciamento OR doença)'

## 2 Definições de Funções

Abaixo está localizado todas as definições de funções que será usado no projeto:

In [3]:
# 1. Consumo de dados com a News API -------------------------------------------------------------------------------------------------

## 1 Usando Biblioteca NewsAPI
def fazer_a_request_1( API_KEY:str='9a77398581d74beebbd29dbebd159a53' , PALAVRAS_CHAVES:str='(genômica OR genômico) AND (terapia OR sequenciamento OR doença)' ):
    # A função realiza o request com API_KEY e PALAVRAS_CHAVES declaradas e é retornado um json
    newsapi = NewsApiClient(api_key=API_KEY)
    response_lib = newsapi.get_everything(q=PALAVRAS_CHAVES,
                                        language='pt',
                                        sort_by='publishedAt'
    )
    return response_lib

## 2 Usando requests
def fazer_a_request_2( API_KEY:str='9a77398581d74beebbd29dbebd159a53' , PALAVRAS_CHAVES:str='(genômica OR genômico) AND (terapia OR sequenciamento OR doença)' ):
    # A função realiza o request com API_KEY e PALAVRAS_CHAVES declaradas e é retornado um json
    url = f'https://newsapi.org/v2/everything?q={PALAVRAS_CHAVES}&language=pt&sortBy=publishedAt&apiKey={API_KEY}'
    response_requests = requests.get(url).json()
    return response_requests



# 2. Definir Critérios de Relevância -------------------------------------------------------------------------------------------------

def tratar_dados(json_de_noticias) -> pd.DataFrame:
    df = pd.json_normalize(json_de_noticias['articles'])
    df['publishedAt'] = pd.to_datetime(df['publishedAt']).dt.tz_localize(None) # inserido, pq linha abaixo estava dando erro "TypeError: Cannot use .astype to convert from timezone-aware dtype to timezone-naive dtype"
    df['publishedAt'] = df['publishedAt'].astype('datetime64[ms]')
    df.fillna("desconhecido", inplace=True)

    # Remover noticias sem informações nas colunas: descrição, conteúdo e nome de fonte 
    df = df[~df['description'].str.contains('desconhecido', case=False)]
    df = df[~df['content'].str.contains('desconhecido', case=False)]
    df = df[~df['source.name'].str.contains('desconhecido', case=False)]

    return df



# 3. Cargas em Batches: ---------------------------------------------------------------------------------------------------------------

def armazenar_noticias(df) -> None:

    # Verifica se já existe um arquivo com os dados
    if os.path.isfile('noticias.csv'):
        # Lê o arquivo existente
        df_existente = pd.read_csv('noticias.csv', index_col=0)

        # Identifica as notícias repetidas
        df_repetidas = df[df['title'].isin(df_existente['title'])]

        # Exclui as notícias repetidas
        df = df.drop(df_repetidas.index)

    # Grava o dataframe em um arquivo CSV
    df.to_csv('noticias.csv', index=False)

    return None

def carrega_armazenamento():
    df = pd.read_csv('noticias.csv')
    return df 



# 4. Dados transformados para consulta do público final ------------------------------------------------------------------------------

def qtd_noticia_ano_mes_dia(df_inteiro): 
# =============================================================================
#     #Alternativa 1
#     # Calcula a quantidade de notícias por ano, mês e dia
#     df_inteiro['ano'] = df_inteiro['publishedAt'].dt.year
#     df_inteiro['mes'] = df_inteiro['publishedAt'].dt.month
#     df_inteiro['dia'] = df_inteiro['publishedAt'].dt.day
# 
#     qtd_por_ano = df_inteiro['ano'].value_counts().sort_index().reset_index()
#     qtd_por_ano.columns = ['ano', 'quantidade']
# 
#     qtd_por_mes = df_inteiro['mes'].value_counts().sort_index().reset_index()
#     qtd_por_mes.columns = ['mes', 'quantidade']
# 
#     qtd_por_dia = df_inteiro['dia'].value_counts().sort_index().reset_index()
#     qtd_por_dia.columns = ['dia', 'quantidade']
# =============================================================================
    
    #Alternativa 2
    # Calcula a quantidade de notícias por ano, mês/ano e dia/mês/ano
    df_inteiro['ano'] = df_inteiro['publishedAt'].dt.year
    df_inteiro['mes_ano'] = df_inteiro['publishedAt'].dt.to_period('M')
    df_inteiro['dia_mes_ano'] = df_inteiro['publishedAt'].dt.to_period('D')
    
    qtd_por_ano = df_inteiro['ano'].value_counts().sort_index().reset_index()
    qtd_por_ano.columns = ['ano', 'quantidade']

    qtd_por_mes = df_inteiro['mes_ano'].value_counts().sort_index().reset_index()
    qtd_por_mes.columns = ['mes_ano', 'quantidade']

    qtd_por_dia = df_inteiro['dia_mes_ano'].value_counts().sort_index().reset_index()
    qtd_por_dia.columns = ['dia_mes_ano', 'quantidade']
    
    return qtd_por_ano, qtd_por_mes, qtd_por_dia 

def qtd_noticia_fonte_autor(df_inteiro):
    # Calcula a quantidade de notícias por fonte
    qtd_por_fonte = df_inteiro['source.name'].value_counts().reset_index()
    qtd_por_fonte.columns = ['fonte', 'quantidade']
    
    # Calcula a quantidade de notícias por autor
    qtd_por_autor = df_inteiro['author'].value_counts().reset_index()
    qtd_por_autor.columns = ['autor', 'quantidade']
    
    return qtd_por_fonte, qtd_por_autor

def qtd_aparicao_palavras_chaves(df_inteiro, palavras_chave):  
    # Adiciona colunas para cada palavra-chave no DataFrame
    for palavra in palavras_chave:
        df_inteiro[palavra] = df_inteiro['description'].str.contains(fr'\b{palavra}\b', case=False)
    
    df_inteiro['ano'] = df_inteiro['publishedAt'].dt.year
    df_inteiro['mes'] = df_inteiro['publishedAt'].dt.month # Aproveitei aqui para deixar as colunas apenas do mês e do dia, pois serão útil na função "qtd_aparicao_palavras_chaves"
    df_inteiro['dia'] = df_inteiro['publishedAt'].dt.day

    # Agrupa por ano, mês e dia e conta a quantidade de aparições de cada palavra-chave
    df_inteiro = df_inteiro.groupby(['ano', 'mes', 'dia'])[palavras_chave].sum().reset_index()
    
    # Cria um dataframe com as colunas 'ano', 'mes', 'dia' e a contagem de cada palavra-chave
    df_contagem_palavras_chave = df_inteiro.melt(id_vars=['ano', 'mes', 'dia'], var_name='palavra_chave', value_name='quantidade')
    
    return df_contagem_palavras_chave   

def armazenar_dados_estatisticos(qtd_por_ano, qtd_por_mes, qtd_por_dia, qtd_por_fonte, qtd_por_autor, df_contagem_palavras_chave):
    print("Quantidade de notícias por ano:")
    print(qtd_por_ano)

    print("\nQuantidade de notícias por mês:")
    print(qtd_por_mes)

    print("\nQuantidade de notícias por dia:")
    print(qtd_por_dia)

    print("\nQuantidade de notícias por fonte:")
    print(qtd_por_fonte)

    print("\nQuantidade de notícias por autor:")
    print(qtd_por_autor)

    print("\nContagem de aparições das palavras-chave:")
    print(df_contagem_palavras_chave)
    
    estatisticas = pd.concat([qtd_por_ano, qtd_por_mes, qtd_por_dia, qtd_por_fonte, qtd_por_autor, df_contagem_palavras_chave], axis=1)
    estatisticas.to_csv('resultado_final.csv', index = False)
    estatisticas.to_excel('resultado_final.xlsx', index = False)
    return None

## 3 Teste

In [4]:
# Instanciar objeto
response = fazer_a_request_1()
# Exibir chaves do dicionário
print(f'Chaves retornadas: {response.keys()}')
# Valores do dicionário
print(f"Status: {response['status']}")
print(f"Total de resultados: {response['totalResults']}")
print(f"Exemplo de artigo: {response['articles'][0]}")

Chaves retornadas: dict_keys(['status', 'totalResults', 'articles'])
Status: ok
Total de resultados: 20
Exemplo de artigo: {'source': {'id': None, 'name': 'Terra.com.br'}, 'author': 'Alma Preta', 'title': 'Nordeste é a região do país com maior ancestralidade da África e do Oriente Médio', 'description': 'Levantamento inédito traz dados sobre a ancestralidade dos brasileiros por estado, com base em um banco de dados de mais de 260 mil DNAs Texto: Redação | Foto: Reprodução', 'url': 'https://www.terra.com.br/nos/nordeste-e-a-regiao-do-pais-com-maior-ancestralidade-da-africa-e-do-oriente-medio,37ce6d627d490dd89afcd004cfede315rw30iu52.html', 'urlToImage': 'https://p2.trrsf.com/image/fget/cf/1200/630/middle/images.terra.com/2023/09/29/2012010853-brasilmapa.jpeg', 'publishedAt': '2023-09-29T13:13:29Z', 'content': 'Devido à sua história repleta de migrações, o Brasil é um dos países mais miscigenados do planeta. E isso se reflete na diversidade presente no DNA da população. As três ancestrali

In [5]:
# prova real que ambos os metodos de request dão o mesmo resultado
print(fazer_a_request_1() == fazer_a_request_2())
df = tratar_dados(response)
df

True


,author,title,description,url,urlToImage,publishedAt,content,source.id,source.name
11,Redação,Este hábito comum pode duplicar o risco de doe...,Todos sabem que o hábito de fumar pode causar ...,https://catracalivre.com.br/saude-bem-estar/es...,https://catracalivre.com.br/cdn-cgi/image/f=au...,2023-09-13 04:00:01,Todos sabem que o hábito de fumar pode causar ...,desconhecido,Catracalivre.com.br
15,desconhecido,O que se sabe sobre a nova onda de covid no Br...,Alta de casos da doença pode estar relacionada...,https://valor.globo.com/brasil/noticia/2023/09...,https://s2-valor.glbimg.com/pOcRs2rEKVjvZxWDKI...,2023-09-01 19:07:09,Os testes positivos para covid-19 aumentaram n...,globo,Globo


## 4 Algoritmo

In [6]:
from newsapi import NewsApiClient
import requests
import time

API_KEY = '9a77398581d74beebbd29dbebd159a53'
PALAVRAS_CHAVES_GERAIS = '(genômica OR genômico) AND (terapia OR sequenciamento OR doença)'
INTERVALO = 3600 # 3600 segundos
i = 0
while True:

    # 1. Consumo de dados com a News API
    response = fazer_a_request_1(API_KEY, PALAVRAS_CHAVES_GERAIS)

    # 2. Definir Critérios de Relevância
    df_agora = tratar_dados(response)

    # 3. Cargas em Batches:
    armazenar_noticias(df_agora) # TODO
    
    # 4. Dados transformados para consulta do público final
    if i==23: # i=23 referente as 24h do dia
        df_inteiro = carrega_armazenamento() # TODO
        df_inteiro["publishedAt"] = pd.to_datetime(df_inteiro["publishedAt"])
        a, b, c = qtd_noticia_ano_mes_dia(df_inteiro) # TODO
        d, e = qtd_noticia_fonte_autor(df_inteiro) # TODO
        f = qtd_aparicao_palavras_chaves(df_inteiro, PALAVRAS_CHAVES_ESPECIFICAS) # TODO
        armazenar_dados_estatisticos(a , b , c, d, e, f) # TODO
        i = 0
        
    else:
        i += 1
        
    time.sleep(INTERVALO)


Quantidade de notícias por ano:
    ano  quantidade
0  2023           5

Quantidade de notícias por mês:
   mes_ano  quantidade
0  2023-09           5

Quantidade de notícias por dia:
  dia_mes_ano  quantidade
0  2023-09-27           1
1  2023-09-28           1
2  2023-09-29           3

Quantidade de notícias por fonte:
                 fonte  quantidade
0               R7.com           2
1         Terra.com.br           1
2  Catracalivre.com.br           1
3   Megacurioso.com.br           1

Quantidade de notícias por autor:
                autor  quantidade
0  Redação Terra Você           1
1               Do R7           1
2   Da Agência Brasil           1
3             Redação           1
4       Pedro Freitas           1

Contagem de aparições das palavras-chave:
    ano  mes  dia   palavra_chave  quantidade
0  2023    9   27  sequenciamento           0
1  2023    9   28  sequenciamento           0
2  2023    9   29  sequenciamento           0
3  2023    9   27         terapia   

KeyboardInterrupt: 